In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [16]:
data_train = pd.read_csv('train.csv')
data_train_len = data_train.shape[0]
data_test = pd.read_csv('test.csv')
y = data_train.pop('Attrition')
data = pd.concat([data_train, data_test])

data = data.drop(columns=['EmployeeID', 'StandardHours', 'Over18', 'EmployeeCount'])
data.replace({'JobRole':{'Admin': 'Administrative'}}, inplace=True)
# for colname in data.select_dtypes("O"):
#     data[colname], _ = data[colname].factorize()
data.replace({'Education':{1:'Below_college', 2:'College', 3:'Bachelor', 4:'Masters', 5:'Doctor'}}, inplace=True)
data = pd.get_dummies(data)
data = data.drop(columns=['Gender_Female','OverTime_No'])

train = data.iloc[:data_train_len, :]
test = data.iloc[data_train_len:, :]

In [17]:
def y_enco(y):
    if y == 'No':
        return 0
    elif y == 'Yes':
        return 1
    else:
        pass

y_encoder = y.apply(y_enco)

In [18]:
from imblearn.over_sampling import ADASYN 
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
# Data Normalization

# ada = ADASYN(random_state=42)
# X_res, y_res = ada.fit_resample(X_norm, y_encorder)
# X_res.shape

X_train, X_test, y_train, y_test = train_test_split(train, y_encoder, test_size=0.1, random_state=42)


scaler = MinMaxScaler(feature_range = (0,1))
scaler.fit(X_train)
X_train_norm = scaler.transform(X_train)


ada = ADASYN(random_state=42)
X_res, y_res = ada.fit_resample(X_train_norm, y_train)
X_res.shape, y_res.shape

((2133, 48), (2133,))

In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV

# # Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 500, stop = 1000, num = 6)]
# # Number of features to consider at every split
max_features = ['sqrt']
# # Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(4, 20, num = 9)]
# max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2]
# Minimum number of samples required at each leaf node
min_samples_leaf = [2, 4]
# # Method of selecting samples for training each tree
bootstrap = [False]
# # Create the random grid
params = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
# params = {'n_estimators': n_estimators,
#             'max_depth': max_depth,
#             'min_samples_split': min_samples_split,
#             'min_samples_leaf': min_samples_leaf}

grid_search = GridSearchCV(estimator = RandomForestClassifier(),
                           param_grid=params,
                           cv = 10,
                           n_jobs=-1,
                           verbose=0,
                           scoring="f1")

# Training
grid_search.fit(X_res, y_res)

# Best Performing Parameter
print('='*20)
print("best params: " + str(grid_search.best_estimator_))
print("best params: " + str(grid_search.best_params_))
print('best score:', grid_search.best_score_)
print('='*20)

best params: RandomForestClassifier(bootstrap=False, max_depth=20, min_samples_leaf=2,
                       n_estimators=1000)
best params: {'bootstrap': False, 'max_depth': 20, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 1000}
best score: 0.9721183227034811


In [20]:
from sklearn.metrics import f1_score

X_val_norm = scaler.transform(X_test)

y_pred = grid_search.predict(X_val_norm)

f1_score(y_test, y_pred)

0.7199999999999999

In [21]:
X_test_norm = scaler.transform(test)

y_pred = grid_search.predict(X_test_norm)

id = range(0,len(y_pred))
y_test = pd.DataFrame()
y_test['Id'] = id
y_test['Predicted'] = y_pred
y_test.to_csv('pred_rf_ad.csv', index = 0)